In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import ast
from sklearn.preprocessing import MinMaxScaler
import matplotlib.cm as cm

# Load the data
df = pd.read_csv('../candidates_with_questions.csv')

In [ ]:
# Process the answers column to get a numeric matrix
def extract_answers(row):
    # Convert string representation of list to actual list
    answers = ast.literal_eval(row['answers'])
    # Create dictionary with questionId as key and value as value
    return {item['questionId']: item['value']/100 for item in answers}

# Apply extraction to each row
candidates_answers = df.apply(extract_answers, axis=1)

# Create a dataframe with all questions
question_ids = set()
for answers in candidates_answers:
    question_ids.update(answers.keys())
question_ids = sorted(question_ids)


In [93]:
# Create feature matrix
X = np.zeros((len(df), len(question_ids)))
for i, answers in enumerate(candidates_answers):
    for j, q_id in enumerate(question_ids):
        if q_id in answers:
            X[i, j] = answers[q_id]


In [94]:
print(X.shape[1])

58


In [95]:

# Build autoencoder model
input_dim = X.shape[1]
encoding_dim = 8  # Number of dimensions for spider chart

# Encoder
input_layer = layers.Input(shape=(input_dim,))
encoded = layers.Dense(32, activation='gelu')(input_layer)
encoded = layers.Dense(16, activation='gelu')(encoded)
encoded = layers.Dense(encoding_dim, activation='sigmoid')(encoded)

# Decoder
decoded = layers.Dense(16, activation='gelu')(encoded)
decoded = layers.Dense(32, activation='gelu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

# Models
autoencoder = models.Model(input_layer, decoded)
encoder = models.Model(input_layer, encoded)

In [96]:
# Compile and train
autoencoder.compile(optimizer='adam', loss='bce')
autoencoder.fit(X, X, epochs=100, batch_size=16, shuffle=True, validation_split=0.2)

encoded_features = encoder.predict(X)

Epoch 1/100
94/94 [==============================] - 1s 2ms/step - loss: 0.6424 - val_loss: 0.5622
Epoch 2/100
94/94 [==============================] - 0s 884us/step - loss: 0.5148 - val_loss: 0.5003
Epoch 3/100
94/94 [==============================] - 0s 875us/step - loss: 0.4856 - val_loss: 0.4935
Epoch 4/100
94/94 [==============================] - 0s 863us/step - loss: 0.4821 - val_loss: 0.4915
Epoch 5/100
94/94 [==============================] - 0s 883us/step - loss: 0.4798 - val_loss: 0.4886
Epoch 6/100
94/94 [==============================] - 0s 788us/step - loss: 0.4751 - val_loss: 0.4829
Epoch 7/100
94/94 [==============================] - 0s 834us/step - loss: 0.4694 - val_loss: 0.4764
Epoch 8/100
94/94 [==============================] - 0s 800us/step - loss: 0.4624 - val_loss: 0.4687
Epoch 9/100
94/94 [==============================] - 0s 817us/step - loss: 0.4564 - val_loss: 0.4646
Epoch 10/100
94/94 [==============================] - 0s 1ms/step - loss: 0.4526 - val_loss: 

In [ ]:
# last_layer = autoencoder.predict(X)

# np.testing.assert_almost_equal(X/2.5, last_layer/2.5, 1)

59/59 [==============================] - 0s 327us/step


AssertionError: 
Arrays are not almost equal to 1 decimals

Mismatched elements: 12723 / 108866 (11.7%)
Max absolute difference: 0.39858258
Max relative difference: 558.84311187
 x: array([[0.1, 0.3, 0.1, ..., 0.4, 0.1, 0.3],
       [0.4, 0.1, 0.1, ..., 0.3, 0.3, 0.3],
       [0. , 0.4, 0. , ..., 0.3, 0. , 0.4],...
 y: array([[6.3e-02, 3.5e-01, 1.9e-01, ..., 2.8e-01, 2.1e-01, 2.6e-01],
       [2.6e-01, 1.2e-01, 2.3e-01, ..., 2.9e-01, 1.5e-01, 1.9e-01],
       [1.7e-03, 4.0e-01, 3.8e-05, ..., 2.2e-01, 1.5e-02, 3.8e-01],...

In [ ]:
display(X)

array([[0.25, 0.75, 0.25, ..., 1.  , 0.33, 0.67],
       [1.  , 0.25, 0.25, ..., 0.67, 0.67, 0.67],
       [0.  , 1.  , 0.  , ..., 0.83, 0.  , 1.  ],
       ...,
       [0.  , 1.  , 0.  , ..., 1.  , 0.5 , 1.  ],
       [0.  , 1.  , 0.  , ..., 0.33, 0.  , 1.  ],
       [0.  , 1.  , 0.  , ..., 0.33, 0.  , 1.  ]])

In [ ]:
display(last_layer)

array([[5.01594365e-01, 8.20250511e-01, 3.05027574e-01, ...,
        6.69746280e-01, 3.27180296e-01, 7.16569006e-01],
       [7.93865204e-01, 4.50132191e-01, 6.63603902e-01, ...,
        7.96929359e-01, 4.81088132e-01, 4.57010657e-01],
       [2.84893392e-03, 9.99554515e-01, 7.43997452e-06, ...,
        6.15544677e-01, 2.43224483e-02, 9.65472519e-01],
       ...,
       [1.94381416e-01, 9.67345774e-01, 9.85036325e-03, ...,
        6.48695707e-01, 1.32606775e-01, 9.25872326e-01],
       [6.08318544e-04, 9.99898612e-01, 2.68577793e-07, ...,
        5.42814434e-01, 1.28183672e-02, 9.78325903e-01],
       [6.52043615e-04, 9.99912381e-01, 1.17925936e-07, ...,
        5.47068596e-01, 1.19907204e-02, 9.80617881e-01]], dtype=float32)

In [ ]:

# Create spider charts
def create_spider_chart(features, candidate_name, party):
    # Number of variables
    N = features.shape[0]
    
    # What will be the angle of each axis in the plot
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Close the loop
    
    # Spider chart setup
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    # Add the feature values
    values = features.tolist()
    values += values[:1]  # Close the loop
    
    # Plot
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=candidate_name)
    ax.fill(angles, values, alpha=0.25)
    
    # Set labels
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels([f'Dimension {i+1}' for i in range(N)])
    
    plt.title(f"{candidate_name} ({party}) Political Profile", size=20)
    plt.tight_layout()
    return fig


In [ ]:
for i in range(min(5, len(df))):
    features = encoded_features[i]
    features = (features - features.min()) / (features.max() - features.min() + 1e-10)
    fig = create_spider_chart(features, df.iloc[i]['firstname'] + ' ' + df.iloc[i]['lastname'], 
                              df.iloc[i]['partyAbbreviation'])
    plt.savefig(f"candidate_{i}_spider.png")
    plt.close()


In [ ]:
correlations = []
for dim in range(encoding_dim):
    dim_corr = {}
    for j, q_id in enumerate(question_ids):
        corr = np.corrcoef(encoded_features[:, dim], X[:, j])[0, 1]
        dim_corr[q_id] = corr
    correlations.append(dim_corr)


In [ ]:
for dim, corr_dict in enumerate(correlations):
    print(f"Dimension {dim+1} most correlated with:")
    top_corr = sorted(corr_dict.items(), key=lambda x: abs(x[1]), reverse=True)[:5]
    for q_id, corr in top_corr:
        print(f"  Question {q_id}: {corr:.3f}")

Dimension 1 most correlated with:
  Question 5264: -0.503
  Question 5248: -0.453
  Question 5206: 0.440
  Question 5266: 0.432
  Question 5250: -0.410
Dimension 2 most correlated with:
  Question 5260: 0.678
  Question 5208: 0.678
  Question 5186: -0.609
  Question 5216: 0.608
  Question 5232: 0.595
Dimension 3 most correlated with:
  Question 5188: 0.682
  Question 5186: 0.669
  Question 5190: 0.607
  Question 5202: 0.589
  Question 5162: 0.577
Dimension 4 most correlated with:
  Question 5200: -0.803
  Question 5262: 0.764
  Question 5260: -0.748
  Question 5220: 0.740
  Question 5214: -0.737
